In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

In [39]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
import re

from util import scraping_from_url, create_songs_dataframe

## 動作確認

In [7]:
url = 'https://www.uta-net.com/search/?Aselect=1&Keyword=%E7%99%BD%E7%9F%B3%E7%B4%AC&Bselect=3&x=35&y=14'
soup = scraping_from_url(url)

In [11]:
# htmlをパースして曲名，楽曲のURL，アーティスト名，作詞，作曲者名を取得
contents = []
contents.append(soup.find_all(href=re.compile('/song/\d+/$')))  # 楽曲URL用
contents.append(soup.find_all(href=re.compile('/song/\d+/$')))  # 楽曲名用
contents.append(soup.find_all(class_=re.compile('td2')))        # アーティスト名
contents.append(soup.find_all(class_=re.compile('td3')))        # 作詞名
contents.append(soup.find_all(class_=re.compile('td4')))        # 作曲者名
contents

[[<a href="/song/249595/">I.D ～EScape from Utopia～</a>,
  <a href="/song/271306/">Justice OR Voice</a>,
  <a href="/song/259456/">DIAMOND DAYS</a>,
  <a href="/song/249596/">Melty Fantasia</a>,
  <a href="/song/259457/">ラスト・アクトレス</a>,
  <a href="/song/234795/">瑠璃色金魚と花菖蒲</a>],
 [<a href="/song/249595/">I.D ～EScape from Utopia～</a>,
  <a href="/song/271306/">Justice OR Voice</a>,
  <a href="/song/259456/">DIAMOND DAYS</a>,
  <a href="/song/249596/">Melty Fantasia</a>,
  <a href="/song/259457/">ラスト・アクトレス</a>,
  <a href="/song/234795/">瑠璃色金魚と花菖蒲</a>],
 [<td class="td2"><a href="https://www.uta-net.com/artist/24648/">EScape</a></td>,
  <td class="td2"><a href="https://www.uta-net.com/artist/26961/">ジェネシス×ネメシス</a></td>,
  <td class="td2"><a href="https://www.uta-net.com/artist/25744/">田中琴葉(種田梨沙)、周防桃子(渡部恵子)、馬場このみ(高橋未奈美)、真壁瑞希(阿部里果)、白石紬(南早紀)</a></td>,
  <td class="td2"><a href="https://www.uta-net.com/artist/24648/">EScape</a></td>,
  <td class="td2"><a href="https://www.uta-net.com/artist/2574

In [29]:
informations = []
for i, content in enumerate(contents):
    tmp_list = []
    for element in content:
        if i == 0:
            tmp_list.append(element.get('href'))  # 楽曲URL
        else:
            tmp_list.append(element.string)
    informations.append(tmp_list)
    
# 楽曲情報データフレームの作成
songs_df = pd.DataFrame({
    'URL': informations[0],
    'SongName': informations[1],
    'Artist': informations[2],
    'Lyricist': informations[3],
    'Composer': informations[4]
})
# URLにホスト名を追加
songs_df.URL = songs_df.URL.apply(lambda x : 'https://www.uta-net.com' + x)

songs_df

,Artist,Composer,Lyricist,SongName,URL
0,EScape,中土智博,モリタコータ,I.D ～EScape from Utopia～,https://www.uta-net.com/song/249595/
1,ジェネシス×ネメシス,矢鴇つかさ(Arte Refact),真崎エリカ,Justice OR Voice,https://www.uta-net.com/song/271306/
2,田中琴葉(種田梨沙)、周防桃子(渡部恵子)、馬場このみ(高橋未奈美)、真壁瑞希(阿部里果)、...,高田暁,rino,DIAMOND DAYS,https://www.uta-net.com/song/259456/
3,EScape,小高光太郎,モリタコータ,Melty Fantasia,https://www.uta-net.com/song/249596/
4,田中琴葉(種田梨沙)、周防桃子(渡部恵子)、馬場このみ(高橋未奈美)、真壁瑞希(阿部里果)、...,R・O・N,中村彼方,ラスト・アクトレス,https://www.uta-net.com/song/259457/
5,白石紬(南早紀),ラムシーニ,中村彼方,瑠璃色金魚と花菖蒲,https://www.uta-net.com/song/234795/


In [36]:
lyrics = []
for url in songs_df.URL:
    page = (scraping_from_url(url))
    lyrics.append(page.find(id='kashi_area').text)
lyrics

['Input された\u3000この衝動が\u3000いつか\u3000描いたVirtual じゃない\u3000温もりで EScape from UtopiaLa, la, la, Love is HopePa, pa, pa, Paradigm ShiftDancing in the Pain Proof of the Life息が詰まるくらい\u3000矛盾が脳内Distortion解答(こたえ)が無いから\u3000探し求め続けてるの囚われの\u3000箱庭から聴こえた歌は\u3000世界を変えるイロ\u3000トリ\u3000ドリ\u3000の感情 Hello\u3000ココロ起動大切な\u3000この想いには\u3000奇跡があふれてる過去\u3000未来\u3000つなぐ瞬間 I know\u3000それは希望後悔はしたくない\u3000ただそれだけで強くなれるLa, la, la, Love is HopePa, pa, pa, Paradigm ShiftDancing in the Pain Proof of the Life変わらないと\u3000思い続けてたProtocol進化の代償？それとも原点へ回帰？理不尽な\u3000シアワセには (No more)自由の意味も\u3000霞んでしまうイマ\u3000すぐ\u3000抜け出して Hello\u3000ユメを見ようざわついた\u3000胸の鼓動が Identityなら理屈じゃ\u3000処理できない I know\u3000それが本能景色さえColorfulに\u3000彩るキモチ伝えたいから自問自答のLabyrinth戸惑いさえムダじゃないと\u3000気づかせてくれた不揃いな願いでも Make me... Make me...Make me... Make me...Accept！Virtualじゃない温もりで EScape from Utopiaイロ\u3000トリ\u3000ドリ\u3000の感情 Hello\u3000ココロ起動大切な\u3000この想いには\u3000奇跡があふれてる過去\u3000未来\u3000つなぐ瞬間 I know\u3000それは希望後悔はしたくない\u3000ただそれだけで強くなれるLa, la, la, Love is HopePa, pa, p

In [38]:
tsumugi_df = create_songs_dataframe(url)
tsumugi_df

,Artist,Composer,Lyricist,SongName,URL,Lyric
0,EScape,中土智博,モリタコータ,I.D ～EScape from Utopia～,https://www.uta-net.com/song/249595/,Input された この衝動が いつか 描いたVirtual じゃない 温もりで EScap...
1,ジェネシス×ネメシス,矢鴇つかさ(Arte Refact),真崎エリカ,Justice OR Voice,https://www.uta-net.com/song/271306/,口に含んだラズベリーの感想酸っぱいと甘いと どっちが正解だ？味の差異くらい 譲り渡したってい...
2,田中琴葉(種田梨沙)、周防桃子(渡部恵子)、馬場このみ(高橋未奈美)、真壁瑞希(阿部里果)、...,高田暁,rino,DIAMOND DAYS,https://www.uta-net.com/song/259456/,マイクをギュって握って 想い込めた (そっと)ダンスブーツで駆け出そう 煌めきの世界その声聴...
3,EScape,小高光太郎,モリタコータ,Melty Fantasia,https://www.uta-net.com/song/249596/,柔らかくセツナを染めて触れようとすると 消えちゃいそう臆病で 儚い燈火(ひかり)チクリと胸を...
4,田中琴葉(種田梨沙)、周防桃子(渡部恵子)、馬場このみ(高橋未奈美)、真壁瑞希(阿部里果)、...,R・O・N,中村彼方,ラスト・アクトレス,https://www.uta-net.com/song/259457/,ほらね ショウが今始まる見守っていてくださいなんとしても演じきるわこの役の最期まで林檎のよう...
5,白石紬(南早紀),ラムシーニ,中村彼方,瑠璃色金魚と花菖蒲,https://www.uta-net.com/song/234795/,瑠璃色金魚は恋い焦がれる凛と咲き誇る花菖蒲吐き出す空気は泡の模様決してあなたの心に届かないの...
